In [1]:
# baseline cnn model for mnist
from numpy import mean
from numpy import std
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold
import tensorflow as tf
import cv2
import os
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD

# load train and test dataset
def load_dataset(db_root):
  # load dataset
  dirs = ['000','001','002','003','004','005','005','006','007','008','009','010','011','012']
  trainX = []
  trainY = []
  for dir in dirs:
    files = os.listdir(db_root+dir)
    for file in files:
      image_path = db_root+dir+'/'+file
      edge_image = cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
      trainX.append(edge_image)
      trainY.append(int(dir))
  trainX = tf.convert_to_tensor(trainX)
  trainY = tf.convert_to_tensor(trainY)
  # reshape dataset to have a single channel
  print(trainX.shape)
  trainX = trainX.reshape((trainX.shape[0], 160, 140, 1))
  print(trainX.shape)
  # one hot encode target values
  trainY = to_categorical(trainY)
  return trainX, trainY


In [3]:
trainX, trainY = load_dataset('features/m1/')

(389, 160, 140)
(389, 160, 140, 1)


In [5]:
c=0
for k in trainY:
    if(k[2]== 1):
        c+=1
print(c)

60


In [2]:

# scale pixels
def prep_pixels(tensor_arr):
  # convert from integers to floats
  train_norm = tensor_arr.astype('float32')
  # normalize to range 0-1
  train_norm = train_norm / 255.0
  # return normalized images
  return train_norm

# define cnn model
def define_model():
  model = Sequential()
  model.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(160, 140, 1)))
  model.add(MaxPooling2D((3, 3)))
  model.add(Flatten())
  model.add(Dense(150, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dense(13, activation='softmax'))
  # compile model
  opt = SGD(learning_rate=0.01, momentum=0.9)
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
  return model

# evaluate a model using k-fold cross-validation
def evaluate_model(dataX, dataY, n_folds=5):
  scores, histories = list(), list()
  # prepare cross validation
  kfold = KFold(n_folds, shuffle=True, random_state=1)
  # enumerate splits
  model = define_model()
  for train_ix, test_ix in kfold.split(dataX):
    # select rows for train and test
    trainX, trainY, testX, testY = dataX[train_ix], dataY[train_ix], dataX[test_ix], dataY[test_ix]
    # fit model
    history = model.fit(trainX, trainY, epochs=7, batch_size=32, validation_data=(testX, testY))
    # evaluate model
    _, acc = model.evaluate(testX, testY, verbose=0)
    print('> %.3f' % (acc * 100.0))
    # stores scores
    scores.append(acc)
    histories.append(history)
  model.save_weights('./model1.pth')
  return scores, histories

# plot diagnostic learning curves
def summarize_diagnostics(histories):
  for i in range(len(histories)):
    # plot loss
    plt.subplot(2, 1, 1)
    plt.title('Cross Entropy Loss')
    plt.plot(histories[i].history['loss'], color='blue', label='train')
    plt.plot(histories[i].history['val_loss'], color='orange', label='test')
    # plot accuracy
    plt.subplot(2, 1, 2)
    plt.title('Classification Accuracy')
    plt.plot(histories[i].history['accuracy'], color='blue', label='train')
    plt.plot(histories[i].history['val_accuracy'], color='orange', label='test')
  plt.show()

# summarize model performance
def summarize_performance(scores):
  # print summary
  print('Accuracy: mean=%.3f std=%.3f, n=%d' % (mean(scores)*100, std(scores)*100, len(scores)))
  # box and whisker plots of results
  plt.boxplot(scores)
  plt.show()

# run the test harness for evaluating a model
def run_test_harness(trainX, trainY):
  # load dataset
  # prepare pixel data
  trainX = prep_pixels(trainX)
  print(trainX.shape)
  # evaluate model
  scores, histories = evaluate_model(trainX, trainY)
  # learning curves
  #summarize_diagnostics(histories)
  # summarize estimated performance
  summarize_performance(scores)

# entry point, run the test harness
run_test_harness(trainX,trainY)

(389, 160, 140)
(389, 160, 140, 1)
(389, 160, 140, 1)
Epoch 1/7
10/10 [==============================] - 2s 110ms/step - loss: 1.6977 - accuracy: 0.5016 - val_loss: 0.7549 - val_accuracy: 0.8077
Epoch 2/7
10/10 [==============================] - 1s 93ms/step - loss: 0.3225 - accuracy: 0.9068 - val_loss: 0.8791 - val_accuracy: 0.7436
Epoch 3/7
10/10 [==============================] - 1s 120ms/step - loss: 0.1076 - accuracy: 0.9614 - val_loss: 0.6905 - val_accuracy: 0.8718
Epoch 4/7
10/10 [==============================] - 1s 117ms/step - loss: 0.0516 - accuracy: 0.9839 - val_loss: 1.1972 - val_accuracy: 0.7692
Epoch 5/7
10/10 [==============================] - 1s 100ms/step - loss: 0.2580 - accuracy: 0.9389 - val_loss: 0.9240 - val_accuracy: 0.8333
Epoch 6/7
 7/10 [====================>.........] - ETA: 0s - loss: 0.3515 - accuracy: 0.9375

KeyboardInterrupt: 

In [ ]:
for 

In [3]:
model = define_model()
model.load_weights('./model1.pth')

In [4]:
testX, testY = load_dataset('features/test/m1/')
print(testX[:1].shape, testY.shape)
model.evaluate(testX, testY)


(1, 160, 140, 1) (138, 13)
5/5 [==============================] - 0s 28ms/step - loss: 249.0469 - accuracy: 0.7899


[249.046875, 0.7898550629615784]

In [5]:
print(model.predict(testX[30:40]), testY[30:40])

1/1 [==============================] - 0s 104ms/step
[[0.000000e+00 0.000000e+00 1.000000e+00 0.000000e+00 0.000000e+00
  0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
  0.000000e+00 0.000000e+00 0.000000e+00]
 [0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
  0.000000e+00 0.000000e+00 1.000000e+00 0.000000e+00 0.000000e+00
  0.000000e+00 0.000000e+00 0.000000e+00]
 [0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
  1.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
  0.000000e+00 0.000000e+00 0.000000e+00]
 [0.000000e+00 0.000000e+00 1.000000e+00 0.000000e+00 0.000000e+00
  0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
  0.000000e+00 0.000000e+00 0.000000e+00]
 [0.000000e+00 0.000000e+00 1.000000e+00 0.000000e+00 0.000000e+00
  0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
  0.000000e+00 0.000000e+00 0.000000e+00]
 [0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
